In [1]:
import os
import pandas as pd 
import tensorflow as tf # deep learning (Kerous)
import numpy as np # for np.expends_dims => helps to wrap any information inside array 
# we need to wrap if we have a single sample in a batch but model is accepting multiple so we need to wrap it in a array

In [2]:
df=pd.read_csv(
os.path.join("Dataset","cyberbullying_tweets.csv")
)

In [9]:
unique_cyberbullying_type=df["cyberbullying_type"].unique()
unique_cyberbullying_type_values=unique_cyberbullying_type.tolist()
unique_cyberbullying_type_values

['not_cyberbullying',
 'gender',
 'religion',
 'other_cyberbullying',
 'age',
 'ethnicity']

In [49]:
#encoding labels
counter=0
for i in unique_cyberbullying_type_values:
    df=df.replace(i,counter)
    counter=counter+1

In [67]:
new_labels=[]
for val in df["cyberbullying_type"]:
    x=[0,0,0,0,0,0]
    x[val]=1
    new_labels.append(x)
new_labels=np.array(new_labels)
df["newLabels"]=new_labels
#     df["cyberbullying_type"]

ValueError: Wrong number of items passed 6, placement implies 1

In [16]:
from tensorflow.keras.layers import TextVectorization 
#Textveco... take a word and convert that into a numeric value

In [36]:
X=df["tweet_text"]
y=df["cyberbullying_type"]


0        0
1        0
2        0
3        0
4        0
        ..
47687    5
47688    5
47689    5
47690    5
47691    5
Name: cyberbullying_type, Length: 47692, dtype: int64

In [26]:
MAX_FEATURES=200000 # number of words in the vocab
vectorizer=TextVectorization(max_tokens=MAX_FEATURES,
                            output_sequence_length=1800, # maximum sentence length we will be considering i.e it woll cap comments with length loger then this
                            output_mode="int") # will map every word to a integer value eg. 1= son,2=moon...


In [27]:
vectorizer.adapt(X.values) ## vectorixer removes the punctuations ie standardize the text

In [28]:
vectorized_text=vectorizer(X.values)

In [29]:
vectorized_text

<tf.Tensor: shape=(47692, 1800), dtype=int64, numpy=
array([[    9,   125,   426, ...,     0,     0,     0],
       [   85,    10, 66058, ...,     0,     0,     0],
       [27166,     3,  2383, ...,     0,     0,     0],
       ...,
       [    6,   867,     4, ...,     0,     0,     0],
       [ 1084,    24,     7, ...,     0,     0,     0],
       [  603,    43,   745, ...,     0,     0,     0]], dtype=int64)>

In [30]:
# Building a pipeline :- Map cache shuffle batch and prefatch
dataset=tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset=dataset.cache()
dataset=dataset.shuffle(160000)
dataset=dataset.batch(16) # create batches with 16 samples each
dataset=dataset.prefetch(8) # help prevent bottlenecks

In [31]:
batch_x,batch_y=dataset.as_numpy_iterator().next()
batch_x.shape

(16, 1800)

In [32]:
train=dataset.take(int(len(dataset)*.7)) # taking 70 % data for training
val  =dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2)) # first skipping initial 70 % we have taken as train then take 20%
test =dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [33]:
len(test) # datches

298

In [34]:
train_generator=train.as_numpy_iterator() # will give an iterator which can be used to get next batch
train_generator.next() # will gice next batch 

(array([[   6,   20,   37, ...,    0,    0,    0],
        [  66,   76,    6, ...,    0,    0,    0],
        [1289,  570,  591, ...,    0,    0,    0],
        ...,
        [   6, 1425,  173, ...,    0,    0,    0],
        [  88,    7, 1247, ...,    0,    0,    0],
        [ 156,    3,  140, ...,    0,    0,    0]], dtype=int64),
 array([4, 1, 1, 2, 3, 3, 1, 0, 5, 1, 1, 1, 2, 4, 2, 5], dtype=int64))

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout,Bidirectional,Embedding,Dense


In [37]:
model=Sequential()
# first create the embedding layer
model.add(Embedding(MAX_FEATURES+1,32)) # one embedding for each word 32 is number of feature in a particular embedding
# adding lstm layer and making it bidirectional so data can flow for and back
# we are using bidirectional because meaning of one word in a sentence may depent on some other word i.e word before or after it 
model.add(Bidirectional(LSTM(32,activation="tanh"))) 
# Feature extraction fully connected layers
model.add(Dense(128.,activation="relu"))
model.add(Dense(256.,activation="relu"))
model.add(Dense(128.,activation="relu"))
# Final Layer  with 6 unit ie one for each of label
model.add(Dense(6,activation="sigmoid"))



In [38]:
model.compile(loss="BinaryCrossentropy",optimizer="Adam")

In [39]:
hitory=model.fit(train,epochs=1,validation_data=val)

ValueError: in user code:

    File "C:\Users\LENOVO\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\LENOVO\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\LENOVO\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\LENOVO\anaconda3\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\LENOVO\anaconda3\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "C:\Users\LENOVO\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    F